In [1]:
# ОПИСАНИЕ ЗАДАНИЯ
#Построй линейный прогноз любого из интересующего тебя пункта отчетности.
#Файлы которые я тебе скину - это реальная отчетность с интерфакса из под нашей OCR.
#Поэтому заодно применишь уже полученные знания в обработке плохо собранных таблиц
#Тебе нужно будет вытащить нужные ячейки, построить из этого график и спрогнозировать куда будет двигаться
#выбранный тобой пункт отчетности с последующими отчетностями.
#Как раз отработаешь уже изученные обработки таблиц и данных, построение графиков,
#методы группировки и скорее всего будешь вынуждена поработать с numpy, если не трогала его еще, как раз познакомишься.
#Ну и конечно задача регрессии машинного обучения что у тебя фактически актуальной темой является

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re

In [3]:
def add_balance(data):
    '''функция для добавления новых столбцов в df_balance'''
    data = data.dropna(how='all')
    for item in data.index:
        if data.iloc[item, 0] == 'Наименование показателя':
            i = item
            break
    new_column_list = data.iloc[i, 2:5].dropna().to_list()
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
    balance = data.iloc[(i+1):(j+1), 2:(2+len(new_column_list))]
    balance = balance.reset_index(drop=True)
    balance.columns = new_column_list
    return balance


def add_assets(data_new):
    '''функция для добавления активов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Внеоборотные активы Нематериальные активы', str(data_new.iloc[item, column])) != None:
                i = item
                c = column
                break
    new_column_list = data_new.iloc[(i-2), c:(c+4)].to_list()
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('сумма строк 1100', str(data_new.iloc[item, column])) != None:
                j = item
                break
    balance1 = data_new.iloc[i:(j), c:(c+4)]
    balance1 = balance1.reset_index(drop=True)
    balance1.columns = new_column_list
    balance1 = balance1.dropna()
    return balance1


def add_liabilities(data_new):
    '''функция для добавления пассивов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Капитал и резервы Уставный капитал', str(data_new.iloc[item, column])) != None:
                m = item
                n = column
                break
    new_column_list = data_new.iloc[(m-2), n:(n+4)].to_list()
    for item in data_new.index:
        if re.search('сумма строк 1300', str(data_new.iloc[item, n])) != None:
            j = item
            break
    balance2 = data_new.iloc[(m):(j+1), n:(n+4)]
    balance2 = balance2.reset_index(drop=True)
    balance2.columns = new_column_list
    balance2 = balance2.dropna()
    return balance2

In [4]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
# files_list = ['712_2509_0.xlsx', '713_2509_0.xlsx', '715_2509_0.xlsx', '716_2509_0.xlsx', '717_2509_0.xlsx', 
                #'719_2509_0.xlsx', '720_2509_0.xlsx', '721_2509_0.xlsx', '723_2509_0.xlsx', '724_2509_0.xlsx', 
                #'725_2509_0.xlsx', '727_2509_0.xlsx', '728_2509_0.xlsx', '729_2509_0.xlsx', '731_2509_0.xlsx', 
                #'732_2509_0.xlsx', '733_2509_0.xlsx', '735_2509_0.xlsx', '736_2509_0.xlsx', '737_2509_0.xlsx']

In [6]:
df_balance = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance = add_balance(data)
    df_balance[balance.columns] = balance

In [7]:
df_balance = df_balance.dropna(axis=1)

In [8]:
df_balance.columns = ['2020.06.30', '2019.12.31', '2020.03.31', '2019.09.30', '2017.12.31', '2018.06.30',
                      '2016.12.31', '2017.09.30', '2015.12.31', '2016.09.30',
                      '2015.12.31', '2014.12.31', '2016.06.30', '2016.03.31',
                      '2015.06.30', '2013.12.31', '2014.06.30', '2012.12.31']

In [9]:
df_balance = df_balance.loc[:, ~df_balance.columns.duplicated()]

In [10]:
df_balance = df_balance.reindex(sorted(df_balance.columns), axis=1)

In [11]:
data = pd.read_excel("./712_2509_0-Copy1.xlsx", header=None)
data = data.dropna(how='all')
for item in data.index:
    if data.iloc[item, 0] == 'Наименование показателя':
        i = item
        break
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
text_column_list = data.iloc[i, 0:2].dropna().to_list()
data = data.iloc[(i+1):(j+1), 0:2]
data = data.reset_index(drop=True)
data.columns = text_column_list
data[df_balance.columns] = df_balance

In [12]:
df_data = data.drop(0)

In [13]:
df_data.reset_index(drop=True, inplace=True)

In [14]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements1\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [15]:
# files_list ['722_2509_0.xlsx', '730_2509_0.xlsx', '734_2509_0.xlsx', '738_2509_0.xlsx', '739_2509_0.xlsx', '740_2509_0.xlsx']
# из 2-3 файлов не удалось совсем вытащить данные, но в итоге того, что есть хватило, т.к. в отчетность обучно включаются показатели
#прошлого и текущаго года.

In [16]:
df_balance1 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_assets(data)
    df_balance1[balance_new.columns] = balance_new

In [17]:
df_balance2 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_liabilities(data)
    df_balance2[balance_new.columns] = balance_new

In [18]:
balance_final = df_balance1.append(df_balance2, ignore_index=True)
balance_f = balance_final.drop(['Наименование показателя', 'Код'], axis=1)
balance_f.columns

Index(['На 31.12.2017', 'На 31.12.2016', 'На 31.12.2015', 'На 31.12.2014',
       'На 31.12.2013', 'На 31.12.2012', 'На 31.12.2011', 'На 31.12.2010'],
      dtype='object')

In [19]:
balance_f.columns = ['2017.12.31', '2016.12.31', '2015.12.31', '2014.12.31', '2013.12.31',
                     '2012.12.31', '2011.12.31', '2010.12.31']

In [20]:
data_final = pd.concat([df_data, balance_f], axis=1)
data_final = data_final.loc[:, ~data_final.columns.duplicated()]
data_final = data_final.reindex(sorted(data_final.columns), axis=1)

In [21]:
data_final = data_final.loc[:, ['Наименование показателя', 'Код', '2010.12.31', '2011.12.31', '2012.12.31', '2013.12.31', '2014.06.30',
                                '2014.12.31', '2015.06.30', '2015.12.31', '2016.03.31', '2016.06.30',
                                '2016.09.30', '2016.12.31', '2017.09.30', '2017.12.31', '2018.06.30',
                                '2019.09.30', '2019.12.31', '2020.03.31', '2020.06.30']]

In [22]:
for column in data_final.columns:
    data_final[column]= data_final[column].replace('—', 0) 
    data_final[column]= data_final[column].replace('-', 0) 

In [23]:
data_final = data_final.T
name_list = data_final.loc['Наименование показателя'].to_list()
data_final = data_final.drop('Наименование показателя')
data_final.columns = name_list
data_final.drop('Код', axis=0)
data_final = data_final.fillna(0)

In [24]:
data_final = data_final.drop('Код', axis=0)

In [25]:
data_final

,А К Т И В I. Внеоборотные активы Нематериальные активы,Результаты исследований и разработок,Основные средства,Финансовые вложения,Отложенные налоговые активы,Прочие внеоборотные активы,Итого по разделу I,II. Оборотные активы Запасы,Налог на добавленную стоимость по приобретенным ценностям,Дебиторская задолженность,...,Итого по разделу III,IV. Долгосрочные обязательства Заемные средства,Отложенные налоговые обязательства,Прочие обязательства,Итого по разделу IV,V. Краткосрочные обязательства Заемные средства,Кредиторская задолженность,Оценочные обязательства,Итого по разделу V,БАЛАНС (сумма строк 1300 + 1400 + 1500)
2010.12.31,235 613,16 740,116 905 617,96 593 737,103456,6125199 4 223 704,219 980 362,33 140 868,4 007 830,35 958 164,...,62 261 647,5 823 641,1 449 345,69 534 633,16 349 819,19 846 130,0,0,36195 949,0
2011.12.31,316421,21 093,144 561 560,92 369 776,31 756,4151 975 1 713 785,241 452 581,44 287 413,7 942 249,71 081 197,...,69425 901,6 862 388,88 766,76 377 055,36 508 193,32 504475,0,877 026,69 889 694,0
2012.12.31,1 748 963,28 411,1 032 642,147 780 380,113 130 556,85 019,861 259,264 667 230,37 661 071,7 934 120,...,310 161 553,320 609 076,61 455 664,6 762 292,936 305,69 154 261,44 513 482,30 248 292,1 007 390,75 769 164
2013.12.31,255 884,37 488,0,142 980 881,132 979 956,106 873,3 504 849,279 865 931,37 631 821,8 063 796,...,293 636 474,304 066 597,90 451 782,8 476 574,158 925,99 087 281,22 336 976,41 868 595,1 744 937,65 950 508
2014.06.30,316 893,60 360,0,139 369 815,138 063 122,115 826,1 909 688,279 835 704,31 396 563,7 681 084,...,307 292 674,317 709 418,80 503 843,9 069 763,163 687,89 737 293,32 454 324,37 164 647,1 443 632,71 062 603
2014.12.31,346 166,30 947,135 334 350,140 632 205,128 932,1 053 135,277 525 735,37 431 161,8 058 219,111 139 719,...,314 725 971,96 384 209,9 786 365,409 869,106 580 443,23 556 046,56 053 228,1 442 257,81 051 531,502 357 945
2015.06.30,441 605,26 633,132 184 057,151 952 921,138 525,1 426 391,286 170 132,41 147 773,9 235 886,107 465 414,...,326 732 209,91 974 675,10 843 985,20 902,102 839 562,22 605 714,63 250 178,2 961 852,88 817 744,518 389 515
2015.12.31,479 344,28 414,131 024 265,152 520 560,148 597,1 846 963,286 048 143,42 543 057,7 914 804,123 496 442,...,328 233 790,137 275 933,11 079 929,22 861,148 378 723,27 893 440,59 011078,1911 146,88 815 664,565 428 177
2016.03.31,669 319,27 154,132 033 566,145 313 809,150 223,2 038 995,280 233 066,37 710 009,6 392 006,130 057 929,...,333 939 566,126 276 389,10 879 663,2 846,137 158 898,30 588 262,48 295 229,3 289 188,82 172 679,553 271 143
2016.06.30,727 720,25 101,131 564 386,145 564 766,152 158,2 092 533,280 126 664,39 925 142,6 019 980,61 185 524,...,330 937 283,128 609 326,10 938 742,2 845,139 550 913,26 011 098,52 955 675,3 383 533,82 350 306,552 838 502
